In [65]:
from instrumented_solvers import *
from test import get_assignment_problems, get_assignment_results, get_solvers, get_test_results

import numpy as np
import pandas as pd

In [27]:
setting_problem_stats_dict = get_test_results()

(min_conflicts_instrumented, max_steps=1000)(australia_map)
(min_conflicts_instrumented, max_steps=1000)(france_map)
(min_conflicts_instrumented, max_steps=1000)(zebra)
(min_conflicts_instrumented, max_steps=1000)(5 queens)
(min_conflicts_instrumented, max_steps=1000)(7 queens)
(min_conflicts_instrumented, max_steps=1000)(9 queens)
(min_conflicts_instrumented, max_steps=1000)(11 queens)
(min_conflicts_instrumented, max_steps=1000)(13 queens)
(backtracking_search_instrumented, select_unassigned_variable=first_unassigned_variable, order_domain_values=unordered_domain_values, inference=no_inference, max_steps=1000)(australia_map)
(backtracking_search_instrumented, select_unassigned_variable=first_unassigned_variable, order_domain_values=unordered_domain_values, inference=no_inference, max_steps=1000)(france_map)
(backtracking_search_instrumented, select_unassigned_variable=first_unassigned_variable, order_domain_values=unordered_domain_values, inference=no_inference, max_steps=1000)(zebra

In [67]:
#create results table

no_iter = 10

backtrack_results_df = pd.DataFrame([])
min_conflicts_results_df = pd.DataFrame([])

algo_parms = list(setting_problem_stats_dict.keys()) #primary keys
for setting in algo_parms:
    
    #figure out what solver you are using 
    solver = setting.split(",")[0][1:]
    results_per_problem = setting_problem_stats_dict[setting]
    problems = list(results_per_problem.keys()) #second level keys
    
    for problem in problems:
        results = results_per_problem[problem]
        stats = list(results.keys()) #third level keys
        
        for stat in stats:
            result_all_iter = results[stat]
                
            if stat == "assignment":
                solutions_found = np.array([not(result is None) for result in result_all_iter])
                prob_of_sucess = (np.sum(solutions_found)/10) * 100

            elif stat == "num_assignments":
                avg_num_assignments = np.mean(result_all_iter)
                sd_num_assignments = np.std(result_all_iter)

            else: #number of repair assignments or backtracks depending on solver

                if solver == 'min_conflicts_instrumented':
                    avg_repair_assignments = np.mean(result_all_iter)
                    sd_repair_assignments = np.std(result_all_iter)
                else:
                    avg_backtracks = np.mean(result_all_iter)
                    sd_backtracks = np.std(result_all_iter)
                    
        #once finished computing stats assign to correct dataframe
        if solver == 'min_conflicts_instrumented':
            row = pd.DataFrame({"problem":problem, 
                               "avg assignments": avg_num_assignments,
                               "sd assignments": sd_num_assignments,
                               "avg repair": avg_repair_assignments,
                               "sd repair":sd_repair_assignments,
                               "prob sucess": prob_of_sucess}, index=range(1))
            min_conflicts_results_df = min_conflicts_results_df.append(row, ignore_index=True)
        else:
            row = pd.DataFrame({"problem":problem, 
                   "avg assignments": avg_num_assignments,
                   "sd assignments": sd_num_assignments,
                   "avg backtracks": avg_backtracks,
                   "sd backtracks":sd_backtracks,
                    "prob sucess": prob_of_sucess}, range(1))
            backtrack_results_df = backtrack_results_df.append(row, ignore_index=True)
                
                          
        
    

In [74]:
#combine outputs
results_df = backtrack_results_df.append(min_conflicts_results_df, ignore_index=True)